## Convexity Demo

Author: Cheuk Ting Li  

In [1]:
from psitip import *
PsiOpts.setting(solver = "pyomo.glpk")  # Set linear programming solver
PsiOpts.setting(repr_latex = True)      # Turn on Jupyter Notebook LaTeX display
PsiOpts.setting(venn_latex = True)      # Turn on LaTeX in diagrams

X, Y, Z, W, U, M, S = rv("X, Y, Z, W, U, M, S") # Declare random variables
R1, R2 = real_array("R", 1, 3)

In [2]:
H(X).isconcave(X)   # H(X) is concave in X

True

In [3]:
# The argument bnet = (X, Y) (the Bayesian network X -> Y) 
# means that the distribution is factorized as p(x)p(y|x)
I(X & Y).isconcave(X, bnet = (X, Y)) # I(X;Y) concave in p(x)

True

In [4]:
I(X & Y).isconvex(Y, bnet = (X, Y)) # I(X;Y) convex in p(y|x)

True

In [5]:
I(X&Y|Z).isaffine(Z, bnet = (Z, X+Y)) # I(X;Y|Z) affine in p(z)

True

------

### Maximization / Minimization Quantities

In [6]:
# Define the information bottleneck [Tishby-Pereira-Bialek 1999]
def info_bot(X, Y, t):
    U = rv("U")
    return (markov(U, X, Y) & (I(X & U) <= t)).maximum(I(Y & U), U)

X, Y = rv("X, Y")
t1, t2 = real_array("t", 1, 3)

# Check that info bottleneck is non-decreasing
bool((t1 <= t2) >> (info_bot(X, Y, t1) <= info_bot(X, Y, t2)))

True

In [7]:
info_bot(X, Y, t1).isconcave()  # info bottleneck concave in t

True

In [8]:
info_bot(X, Y, t1).isconvex()  # info bottleneck not convex in t

False

------

### Upper Concave Envelope

In [9]:
(I(X & Y) - I(X & Z)).isconcave(X+Y, bnet = [X+Y, (X+Y, Z)]) # not concave

False

In [10]:
# Compute upper concave envelope with respect to p(x,y) [Nair 2013]
env = (I(X & Y) - I(X & Z)).concave_envelope(X+Y, bnet = [X+Y, (X+Y, Z)])
env

(( ( markov(Z, X+Y, Q) ) ).exists(Q)).maximum((I(X&Y|Q)-I(X&Z|Q)))

In [11]:
env.isconcave(X+Y, bnet = [X+Y, (X+Y, Z)]) # it is concave

True

------

### Convexity of Regions

In [12]:
# Multiple access channel region without convexification
r = (indep(X, Y) & (R1 >= 0) & (R2 >= 0) & (R1 <= I(X & Z | Y)) 
     & (R2 <= I(Y & Z | X)) & (R1+R2 <= I(X+Y & Z)))
r

( ( R1 >= 0 )
 &( R2 >= 0 )
 &( R1 <= I(X&Z|Y) )
 &( R2 <= I(Y&Z|X) )
 &( R1+R2 <= I(X+Y&Z) )
 &( indep(X, Y) ) )

In [13]:
r.isconvex(X+Y, bnet = [(X, Z), (Y, Z)])  # not convex in p(x,y)

False

In [14]:
rc = r.convexified(X+Y, bnet = [(X, Z), (Y, Z)]) # convexify the region
rc

( ( R1 >= 0 )
 &( R2 >= 0 )
 &( R1 <= I(X&Z|Y+Q) )
 &( R2 <= I(Y&Z|X+Q) )
 &( R1+R2 <= I(X+Y&Z|Q) )
 &( markov(X, Q, Y) )
 &( markov(Q, X+Y, Z) ) ).exists(Q)

In [15]:
rc.isconvex(X+Y, bnet = [(X, Z), (Y, Z)]) # new region convex in p(x,y)

True

### References
- N. Tishby, F. C. Pereira, and W. Bialek, "The information bottleneck method," arXiv preprint physics/0004057
- C. Nair, "Upper concave envelopes and auxiliary random variables," International Journal of Advances in Engineering Sciences and Applied Mathematics, vol. 5, no. 1, pp. 12-20, 2013.
-  R. Ahlswede, "Multi-way communication channels," in 2nd Int. Symp. Inform. Theory, Tsahkadsor, Armenian SSR, 1971, pp. 23–52.
- H. Liao, "Multiple access channels," Ph.D. dissertation, University of Hawaii, Honolulu, HI, 1972.
- R. Ahlswede, "The capacity region of a channel with two senders and two receivers," The annals of probability, vol. 2, no. 5, pp. 805–814, 1974
